In [133]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

In [305]:
opts=Options()

In [306]:
driver = webdriver.Chrome(options=opts,executable_path=r"C:\Users\Deborishi\Downloads\chromedriver.exe")

In [307]:
url='https://www.propertiesguru.com/residential-search/2bhk-residential_apartment_flat-for-sale-in-new_delhi'

In [308]:
driver.get(url)

## Function to infinite load the page

In [309]:
def page():    
    time.sleep(2)
    scroll_pause_time=1
    screen_height = driver.execute_script("return window.screen.height;")
    i=1
    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height,i=i))
        i+=1
        time.sleep(scroll_pause_time)
        scroll_height=driver.execute_script("return document.body.scrollHeight;")
        if (screen_height)*i>scroll_height:
            break
    soup=BeautifulSoup(driver.page_source,"html.parser")
    title=soup.find_all('div', class_='filter-property-list detailurl')
    return title

## Function to select the 3 and 4 specifically numbered BHK flats

In [178]:
def third_fourth():
    #to uncheck 2bhk properties
    two=driver.find_element_by_xpath('//*[@id="2"]')
    #to check 3bhk properties
    three=driver.find_element_by_xpath('//*[@id="3"]')
    #to check 4bhk properties
    four=driver.find_element_by_xpath('//*[@id="4"]')
    driver.implicitly_wait(10)
    #perform the click part
    ActionChains(driver).move_to_element(two).click(two).perform()
    ActionChains(driver).move_to_element(three).click(three).perform()
    ActionChains(driver).move_to_element(four).click(four).perform()

In [179]:
third_fourth()

In [312]:
a=page()

## Function to prepare the data

In [318]:
def dat_prep(a):    
    flats=[]
    price=[]
    area=[]
    facing=[]
    status=[]
    feature1=[]
    feature2=[]
    feature3=[]
    feature4=[]
    owner=[]
    when_posted=[]
    for each in a:
        flats.append(each.find_all('h1', class_='filter-pro-heading')[0].text)
        price.append(each.find_all('span',class_='price')[0].text)
        area.append(each.find_all('div',class_='col-4')[0].text)
        facing.append(each.find_all('div',class_='col-3')[1].text)
        status.append(each.find_all('div',class_='col-5')[0].text)
        re=each.find_all('ul',class_='pro-list')
        li=re[0].find_all('li')
        feature1.append(li[0].text)
        feature2.append(li[1].text)
        feature3.append(li[2].text)
        feature4.append(li[3].text)
        owner.append(each.find_all('span',class_='owner-name')[0].text)
        when_posted.append(each.find_all('span',class_='owner-post')[0].text)    
    return flats,price,area,facing,status,feature1,feature2,feature3,feature4,owner,when_posted
flats,price,area,facing,status,feature1,feature2,feature3,feature4,owner,when_posted=dat_prep(a)

In [315]:
url_page=pd.DataFrame({'Flat type and location': flats,'price':price,'area':area,'side-facing':facing,'status':status,'feature 1':feature1,'feature 2':feature2,'feature 3':feature3,'feature 4':feature4,'owner':owner,'date posted':when_posted})

In [317]:
url_page.to_csv('initial page with bhk details.csv')